In [ ]:
"""
This file generates tables that show how the upper bound can enable early sopping of ASW based optimisation loops.

Notes: 
    - The unlabeled datasets are available at https://archive.ics.uci.edu/
    - The labeled datasets are available at https://github.com/deric/clustering-benchmark/tree/master/src/main/resources/datasets/real-world 
"""

In [1]:
import os 
import numpy as np
import utils 

In [2]:
logger = utils.get_logger(__name__)

In [ ]:
def table_row(data: np.ndarray, dataset: str, k_range: range = range(2, 16), epsilon: float = 0.15, metric: str = "euclidean"):

    logger.info(f"\nDistance metric: {metric}")

    n = data.shape[0]

    if n <= 300:
        k_range = range(2, 31)
        epsilon = 0.35

    ub_dict = utils.get_upper_bound(data=data, metric=metric)

    dissimilarity_matrix = utils.data_to_distance_matrix(data=data, metric=metric)

    kmeans_dict = utils.asw_optimization(algorithm=utils.algorithm_kmeans,
                        data=data,
                        k_range=k_range,
                        asw_metric=metric,
                        ub_reference=ub_dict["ub"],
                        epsilon=epsilon
                    )

    kmedoids_dict = utils.asw_optimization(algorithm=utils.algorithm_kmedoids,
                                             data=dissimilarity_matrix,
                                             k_range=k_range,
                                             asw_metric="precomputed"
                                             )
    
    kmeans_wcre = (ub_dict["ub"] - kmeans_dict["best_score"]) / ub_dict["ub"]
    kmedoids_wcre = (ub_dict["ub"] - kmedoids_dict["best_score"]) / ub_dict["ub"]

    return [
        dataset,
        str(k_range),
        str(epsilon),
        f"${kmeans_dict['best_score']:.3f}$",
        f"${kmedoids_dict['best_score']:.3f}$",
        ub_dict["ub"],
        f"{len(utils.Counter(kmeans_dict['best_labels']))}",
        f"{len(utils.Counter(kmedoids_dict['best_labels']))}",
        kmeans_wcre,
        kmedoids_wcre,
        str(kmeans_dict["stopped_early"]),
        str(kmedoids_dict["stopped_early"]),
    ]

In [8]:
def table(dataset_list: list, data_type: str = "unlabeled"):
    """
    Print table in terminal.
    """

    headers = [
        "Dataset",
        "K cand.",
        "epsilon",
        "Best ASW Kmeans",
        "Best ASW Kmedoids",
        "UB",
        "Best K Kmeans",
        "Best K Kmedoids",
        "Worst case relative error Kmeans",
        "Worst case relative error Kmedoids",
        "Early stop Kmeans",
        "Early stop Kmeans",
    ]

    lines = []

    # Format header
    header_line = "| " + " | ".join(headers) + " |"
    lines.append(header_line)
    separator = "| " + " | ".join(["---"] * len(headers)) + " |"
    lines.append(separator)

    for dataset in dataset_list:

        if data_type == "unlabeled":
            if dataset == "conference_papers":
                data = utils.load_unlabeled_data(dataset=dataset, transpose=True)
            else:
                data = utils.load_unlabeled_data(dataset=dataset)
            row = table_row(data=data, dataset=dataset)

        elif data_type == "labeled":
            data = dataset["X"]
            row = table_row(data=data, dataset=dataset["name"])

        lines.append(
            " & ".join(
                f"${cell:.3f}$" if type(cell) is not str else f"{cell}" for cell in row
            )
            + " \\\ "
        )

    # Print table to terminal
    print("\nTABLE\n")
    for line in lines:
        print(line)

In [9]:
# -------------------------------------------------
# Unlabeled Datasets
# -------------------------------------------------
dataset_list = [
        "religious_texts",
        "ceramic", 
        "conference_papers", 
        "rna", 
    ]

table(dataset_list=dataset_list)

2025-09-02 19:54:34 | utils | INFO | ==== Running dataset: religious_texts ====

2025-09-02 19:54:35 | utils | INFO | Data shape: (590, 8266)
2025-09-02 19:54:35 | utils | INFO | Data shape (zeros removed): (589, 8266)
2025-09-02 19:54:35 | __main__ | INFO | 
Distance metric: euclidean
2025-09-02 19:54:35 | utils | INFO | Computing upper bound
2025-09-02 19:54:35 | utils | INFO | UB: 0.8463322667605195
2025-09-02 19:54:35 | utils | INFO | Optimizing ASW
100%|██████████| 2/2 [00:00<00:00,  7.12it/s]
2025-09-02 19:54:35 | utils | INFO | Optimizing ASW
100%|██████████| 2/2 [00:07<00:00,  3.60s/it]
2025-09-02 19:54:43 | utils | INFO | ==== Running dataset: ceramic ====

2025-09-02 19:54:43 | utils | INFO | Data shape: (88, 17)
2025-09-02 19:54:43 | utils | INFO | Data shape (zeros removed): (88, 17)
2025-09-02 19:54:43 | __main__ | INFO | 
Distance metric: euclidean
2025-09-02 19:54:43 | utils | INFO | Computing upper bound
2025-09-02 19:54:43 | utils | INFO | UB: 0.8549557176260866
2025-0


TABLE

| Dataset | K cand. | epsilon | Best ASW Kmeans | Best ASW Kmedoids | UB | Best K Kmeans | Best K Kmedoids | Worst case relative error Kmeans | Worst case relative error Kmedoids | Early stop Kmeans | Early stop Kmeans |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
religious_texts & range(2, 4) & 0.15 & $0.428$ & $0.846$ & $0.846$ & 3 & 2 & $0.495$ & $0.000$ & False & False \\ 
ceramic & range(2, 4) & 0.35 & $0.584$ & $0.584$ & $0.855$ & 2 & 2 & $0.317$ & $0.317$ & True & False \\ 
conference_papers & range(2, 4) & 0.15 & $0.067$ & $0.384$ & $0.439$ & 2 & 2 & $0.847$ & $0.126$ & False & False \\ 
rna & range(2, 4) & 0.15 & $0.167$ & $0.187$ & $0.419$ & 3 & 3 & $0.601$ & $0.555$ & False & False \\ 


In [10]:
# -------------------------------------------------
# Labeled Datasets
# -------------------------------------------------
dataset_dir = "data/labeled/real_world"

datasets = []
for fname in os.listdir(dataset_dir):
    if fname.endswith(".arff"):
        try:
            path = os.path.join(dataset_dir, fname)
            _, X, _ = utils.load_arff_as_distance_matrix(path, scale=True)

            datasets.append({
                "name": fname.replace(".arff", ""),
                "X": X,
            })
        except:
            continue

dataset_list = datasets[:20]  # pick a subset

logger.info("Datasets processed!")

2025-09-02 19:55:48 | __main__ | INFO | Datasets processed!


In [11]:
table(dataset_list=dataset_list, data_type="labeled")

2025-09-02 19:55:50 | __main__ | INFO | 
Distance metric: euclidean
2025-09-02 19:55:50 | utils | INFO | Computing upper bound
2025-09-02 19:55:50 | utils | INFO | UB: 0.7079522637070917
2025-09-02 19:55:50 | utils | INFO | Optimizing ASW
100%|██████████| 2/2 [00:00<00:00, 20.22it/s]
2025-09-02 19:55:50 | utils | INFO | Optimizing ASW
100%|██████████| 2/2 [00:13<00:00,  6.79s/it]
2025-09-02 19:56:04 | __main__ | INFO | 
Distance metric: euclidean
2025-09-02 19:56:04 | utils | INFO | Computing upper bound
2025-09-02 19:56:04 | utils | INFO | UB: 0.8538590675108088
2025-09-02 19:56:04 | utils | INFO | Optimizing ASW
100%|██████████| 2/2 [00:00<00:00, 89.15it/s]
2025-09-02 19:56:04 | utils | INFO | Optimizing ASW
100%|██████████| 2/2 [00:03<00:00,  1.61s/it]
2025-09-02 19:56:07 | __main__ | INFO | 
Distance metric: euclidean
2025-09-02 19:56:07 | utils | INFO | Computing upper bound
2025-09-02 19:56:07 | utils | INFO | UB: 0.6260440460708323
2025-09-02 19:56:07 | utils | INFO | Optimizing


TABLE

| Dataset | K cand. | epsilon | Best ASW Kmeans | Best ASW Kmedoids | UB | Best K Kmeans | Best K Kmedoids | Worst case relative error Kmeans | Worst case relative error Kmedoids | Early stop Kmeans | Early stop Kmeans |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
wdbc & range(2, 4) & 0.15 & $0.345$ & $0.661$ & $0.708$ & 2 & 2 & $0.513$ & $0.067$ & False & False \\ 
ecoli & range(2, 4) & 0.15 & $0.381$ & $0.836$ & $0.854$ & 3 & 2 & $0.554$ & $0.021$ & False & False \\ 
wine & range(2, 4) & 0.35 & $0.285$ & $0.286$ & $0.626$ & 3 & 3 & $0.545$ & $0.543$ & False & False \\ 
wisc & range(2, 4) & 0.15 & $0.574$ & $0.574$ & $0.844$ & 2 & 2 & $0.320$ & $0.320$ & False & False \\ 
iono & range(2, 4) & 0.15 & $0.270$ & $0.413$ & $0.691$ & 2 & 2 & $0.609$ & $0.402$ & False & False \\ 
zoo & range(2, 4) & 0.35 & $0.324$ & $0.337$ & $0.839$ & 3 & 3 & $0.613$ & $0.598$ & False & False \\ 
iris & range(2, 4) & 0.35 & $0.580$ & $0.580$ & $0.878$ & 2 & 2 & $0.339$